# Velocity Data Download Script for Validation
#### Working Script to pull large velocity datasets and later compare to ICESAT2-derived velocities

ICESat-2 hackweek  
June 15, 2020  
Lynn Kaluzienski

# Import necessary modules

In [1]:
import os,re,h5py
import requests
import zipfile

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import scipy, sys, os, pyproj, glob
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import pointCollection as pc
import math

# Import some of the scripts that we have written
import sys
sys.path.append("/home/jovyan/surface_velocity/scripts")
from loading_scripts import atl06_to_dict

# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

# Import MEaSUREs Velocity Datasets

In [3]:
#From Ben Smith's code loading in .tif file, running into issues likely with directories
data_root='/srv/shared/surface_velocity/FIS_Velocity/'
#spatial_extent = np.array([-102, -76, -98, -74.5])
spatial_extent = np.array([-65, -86, -55, -81])
lat=spatial_extent[[1, 3, 3, 1, 1]]
lon=spatial_extent[[2, 2, 0, 0, 2]]
print(lat)
print(lon)
# project the coordinates to Antarctic polar stereographic
xy=np.array(pyproj.Proj(3031)(lon, lat))
# get the bounds of the projected coordinates 
XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
#Originally tried to load data from a local directory, should change to shared directory
Measures_vx=pc.grid.data().from_geotif(os.path.join(data_root,'Measures2_FIS_Vx.tif'), bounds=[XR, YR])
Measures_vy=pc.grid.data().from_geotif(os.path.join(data_root,'Measures2_FIS_Vy.tif'), bounds=[XR, YR])

[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]


In [13]:
# Load a line and plot

data_root='/srv/shared/surface_velocity/'
field_dict={None:['delta_time','latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}

rgt = "0848"
#rgt = "0537"
cycle="04"
filename = glob.glob(os.path.join(data_root, 'FIS_ATL06', f'*ATL06_*_{rgt}{cycle}*_003*.h5'))[0]

D=atl06_to_dict(filename,'/gt3r', field_dict=field_dict, index=None, epsg=3031)




#Plot Original MEaSUREs Maps of Velocity Components

In [14]:
# show the velocity maps:
plt.figure(figsize=(8,4))
plt.subplot(121)
Measures_vx.show(cmap='viridis', clim=[-100,100])
plt.plot(xy[0,:], xy[1,:],'k')
plt.title('Measures X-Velocity')
plt.plot(D['x'],D['y'],'r')
plt.gca().set_aspect('equal')

plt.subplot(122)
Measures_vy.show(cmap='viridis', clim=[-100,100])
plt.plot(xy[0,:], xy[1,:],'k')
plt.title('Measures Y-Velocity')
plt.plot(D['x'],D['y'],'r')
plt.gca().set_aspect('equal')

#plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'cmap': 'viridis', 'clim': [-100, 100], 'extent': array([-887878.65809562, -400567.81840096,  200333.90920048,
        561654.87344315]), 'origin': 'lower'}
{'cmap': 'viridis', 'clim': [-100, 100], 'extent': array([-887878.65809562, -400567.81840096,  200333.90920048,
        561654.87344315]), 'origin': 'lower'}


# Interpolate the Measures velocities along track and rotate Velocities

In [16]:
vx = Measures_vx.interp(D['x'],D['y'])
vy = Measures_vy.interp(D['x'],D['y'])

#Solve for angle to rotate Vy to be along track and Vx to be across track
import math
xL=abs((D['x'][0])-(D['x'][1]))
yL=abs((D['y'][0])-(D['y'][1]))

#decides if is descending or ascending path
if D['x'][0]-D['x'][1] < 0:

    theta_rad=math.atan(xL/yL)
    #theta_deg=theta_rad*180/math.pi
    v_along=vy/math.cos(theta_rad)
    v_across=vx/math.cos(theta_rad)

else:
    
    theta_rad=math.atan(xL/yL)
    #theta_deg=theta_rad*180/math.pi
    v_along=vy/math.sin(theta_rad)
    v_across=vx/math.sin(theta_rad)

plt.figure(figsize=(8,4))
plt.subplot(221)
plt.plot(D['x_atc'],vx)
plt.title('MEaSUREs Vx')
plt.subplot(222)
plt.plot(D['x_atc'],vy)
plt.title('MEaSUREs Vy')
plt.subplot(223)
plt.plot(D['x_atc'],v_along)
plt.title('V_along')
plt.subplot(224)
plt.plot(D['x_atc'],v_across)
plt.title('V_across')
plt.tight_layout()

#plt.figure(figsize=(8,4))
#plt.subplot(121)
#plt.plot(D['x_atc'],v_along)
#plt.title('V_along')
#plt.subplot(122)
#plt.plot(D['x_atc'],v_across)
#plt.title('V_across')

# Double check Velocities, v_along should be similar to vy but should major differences where flow angle changes
Vdiff=vy-v_along


plt.figure(figsize=(4,4))
plt.plot(D['x_atc'],Vdiff)
plt.title('Difference between Vy and Valong')

plt.tight_layout()

np.save(('/home/jovyan/surface_velocity/contributors/joseph_martin/v_along%s.npy' % rgt), v_along)
np.save(('/home/jovyan/surface_velocity/contributors/joseph_martin/v_across%s.npy' % rgt), v_across)
np.save(('/home/jovyan/surface_velocity/contributors/joseph_martin/x_atc%s.npy' % rgt), D['x_atc'])

#load rotated velocities into dictionary
#D['v_along']=v_along

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
v_along

array([8.07719855, 8.06396435, 8.04684796, ..., 2.68306995, 2.67934642,
       2.67459118])